In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO
import random

import pysam
import h5py

from Bio import motifs
from Bio import pairwise2
from Bio.Seq import Seq

In [ ]:
import sys

sys.path.insert(0, "/home1/smaruj/akita_utils/")

# from akita_utils import *
import akita_utils

In [ ]:
genome_open = pysam.Fastafile("/project/fudenber_735/genomes/mm10/mm10.fa")

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Tensorflow: ", tf.__version__)

from basenji import dataset, seqnn, dna_io, stream

In [ ]:
# NOTE
# head_i = 0 #human
# head_i = 1 #mouse
#

head_i = 1 #mouse
# head_i = 0
model_num = 1 #which fold to use


#base_dir = '/project/fudenber_735/backup/DNN_HiC/human-mouse_5-16-21/'
#model_dir = base_dir+"/f"+str(model_num)+"_c0/train/"

base_dir = "/project/fudenber_735/tensorflow_models/akita/v2/models/"
model_dir = base_dir + "/f" + str(model_num) + "c0/train/"
model_file  = model_dir + "/model" + str(head_i) + "_best.h5"


# model_dir = '/home1/fudenber/repositories/basenji/manuscripts/akita/'
# model_file = model_dir+'/model_best.h5'

params_file = model_dir + "/params.json"
# params_file -> json (dict) with model's parameters
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params["model"]
    params_train = params["train"]
seq_length = params_model["seq_length"]
params_model["verbose"] = False

seqnn_model = seqnn.SeqNN(params_model)
print("built")

seqnn_model.restore(model_file, head_i=head_i)    # model with the mouse head
print("restored")

In [ ]:
hic_diags = params_model["diagonal_offset"]
try:
    target_crop = params_model["trunk"][-2]["cropping"]
except:
    target_crop = params_model["target_crop"]

print("hic_diags: ", hic_diags) 
print("target_crop: ", target_crop)
print("seq_length: ", seq_length)

target_length_cropped = int((seq_length//2048 - target_crop*2 - hic_diags) * ((seq_length//2048 - target_crop*2 - hic_diags) +1)/2) 
target_map_size = seq_length//2048  - target_crop*2 
triu_tup = np.triu_indices(target_map_size, 2)    # Return the indices for the upper-triangle of an (n, m) array, here k=2 (diagonal offset)
# target_map_size, target_length_cropped, triu_tup[0].shape

print("target_length_cropped: ", target_length_cropped)
print("target_map_size: ", target_map_size)
print("shape of triu_tup[0]: ", triu_tup[0].shape)

In [ ]:
background_file = base_dir + '../analysis/background_seqs.fa'
background_seqs = []
with open(background_file,'r') as f:
  for line in f.readlines():
    if '>' in line: continue
    background_seqs.append(dna_io.dna_1hot(line.strip())) 

In [ ]:
seq_coords_df = akita_utils.prepare_insertion_tsv(
    h5_dirs = '/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5',
    score_key = 'SCD',
    # flank_pad = 60, #how much flanking sequence around the sites to include
    weak_thresh_pct = 1, # don't use sites weaker than this, might be artifacts
    weak_num = 20 ,
    strong_thresh_pct = 99, # don't use sites weaker than this, might be artifacts
    strong_num = 100 ,
    save_tsv=None, # optional filename to save a tsv
)

In [ ]:
strong_seq_coords_df = seq_coords_df[:100]

In [ ]:
# strong_seq_coords_df["end"][0] - strong_seq_coords_df["start"][0]

In [ ]:
# >MA0139.1	CTCF
# A  [    87    167    281     56      8    744     40    107    851      5    333     54     12     56    104    372     82    117    402 ]
# C  [   291    145     49    800    903     13    528    433     11      0      3     12      0      8    733     13    482    322    181 ]
# G  [    76    414    449     21      0     65    334     48     32    903    566    504    890    775      5    507    307     73    266 ]
# T  [   459    187    134     36      2     91     11    324     18      3      9    341      8     71     67     17     37    396     59 ]

In [ ]:
fh = open("CTCF.txt")
for m in motifs.parse(fh, "jaspar"):
    motif = m

pssm = motif.pssm
motif.consensus

In [ ]:
flank = 5000

score_list = []
position_list = []

for i in [j for j in range(len(strong_seq_coords_df))]:
    try:
        scores_here = []
        positions_here = []
        
        TAD_boundary = Seq(genome_open.fetch(strong_seq_coords_df["chrom"][i], strong_seq_coords_df["start"][i]-flank, strong_seq_coords_df["end"][i]+flank).upper())
        for position, score in pssm.search(TAD_boundary, threshold=10.0):
            scores_here.append(score)
            positions_here.append(position)
        
        score_list.append(scores_here)
        position_list.append(positions_here)
    except:
        continue

In [ ]:
df = pd.DataFrame(list(zip(score_list, position_list)), columns =['score', 'position'], index=strong_seq_coords_df.index)
new = strong_seq_coords_df.join(df, how="left")

In [ ]:
new["nr_CTCFs"] = new["score"].apply(lambda x: len(x))

In [ ]:
new

In [ ]:
# print(len(score_list))
plt.figure(figsize=(12,8))
# sns.histplot(data=score_list)

ax = sns.histplot(data=new, x="nr_CTCFs")
ax.set(xlabel='# CTCFs', ylabel='count')
# plt.savefig("SINEB2_scores_hist.png")
plt.show()

In [ ]:
twoCTCFs = new[new["nr_CTCFs"] == 2]

In [ ]:
twoCTCFs["distance"] = twoCTCFs["position"].apply(lambda x: abs(x[0] - x[1]))
twoCTCFs.index = [i for i in range(len(twoCTCFs))]

In [ ]:
twoCTCFs

In [ ]:
# print(len(score_list))
plt.figure(figsize=(12,8))
# sns.histplot(data=score_list)

ax = sns.histplot(data=twoCTCFs, x="distance")
ax.set(xlabel='distance between CTCFs', ylabel='count')
# plt.savefig("SINEB2_scores_hist.png")
plt.show()

In [ ]:
# best_score = 0
# for align in pairwise2.align.localxx(consensus, test):
#     if align.score > best_score:
#         best_score = align.score
# print(best_score)

In [ ]:
# df = pd.DataFrame(list(zip(score_list, position_list, length_list)), columns =["score", "position", "length"])

In [ ]:
# print(len(score_list))
# plt.figure(figsize=(12,8))
# # sns.histplot(data=score_list)

# ax = sns.histplot(data=df, x="score")
# ax.set(xlabel='score', ylabel='count')
# # plt.savefig("SINEB2_scores_hist.png")
# plt.show()

In [ ]:
# plt.figure(figsize=(12,8))
# # sns.histplot(data=score_list)

# ax = sns.histplot(data=df, x="position")
# ax.set(xlabel='position', ylabel='count')
# # plt.savefig("SINEB2_scores_hist.png")
# plt.show()

In [ ]:
# NOT RIGHT

# def directional_position(row):
#     if row["position"] > 0:
#         return row["position"]
#     else:
#         return row["length"] + row["position"]

# df["dir_position"] = df.apply(directional_position, axis=1)

In [ ]:
# df["abs_position"] = df["position"].apply(lambda x: abs(x))
# df["strand"] = df["position"].apply(lambda x: np.sign(x))
# df

In [ ]:
# plt.figure(figsize=(12,8))
# # sns.histplot(data=score_list)

# ax = sns.histplot(data=df, x="abs_position", hue="strand")
# ax.set(xlabel='position', ylabel='count')
# # plt.savefig("SINEB2_scores_hist.png")
# plt.show()

In [ ]:
# i = 3220
# Position = 51
# Scoretest = Seq(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i], SINEB2["end"][i]).upper())

# print("Concensus: TGGCCACCAGGGGGCGCTA")
# if Position > 0:
#     print("Found: ", Scoretest[Position:Position+19])
# else:
#     print("Found: ", Scoretest[Position:Position-19].reverse_complement())

In [ ]:
# SINEB2.iloc[3220]

In [ ]:
# flank = 0
# trimmed_SINEB2_CTCF = Seq(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i] + Position - flank, SINEB2["start"][i] + Position + 19 + flank).upper())
# insert_length = len(trimmed_SINEB2_CTCF)

In [ ]:
# flank = 50
# trimmed_SINEB2_CTCF = Seq(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i], SINEB2["end"][i]).upper())
# insert_length = len(trimmed_SINEB2_CTCF)

In [ ]:
# insert_length

In [ ]:
# NOTE
# head_i = 0 #human
# head_i = 1 #mouse
#

head_i = 1 #mouse
# head_i = 0
model_num = 1 #which fold to use


#base_dir = '/project/fudenber_735/backup/DNN_HiC/human-mouse_5-16-21/'
#model_dir = base_dir+"/f"+str(model_num)+"_c0/train/"

base_dir = "/project/fudenber_735/tensorflow_models/akita/v2/models/"
model_dir = base_dir + "/f" + str(model_num) + "c0/train/"
model_file  = model_dir + "/model" + str(head_i) + "_best.h5"


# model_dir = '/home1/fudenber/repositories/basenji/manuscripts/akita/'
# model_file = model_dir+'/model_best.h5'

params_file = model_dir + "/params.json"
# params_file -> json (dict) with model's parameters
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params["model"]
    params_train = params["train"]
seq_length = params_model["seq_length"]
params_model["verbose"] = False

seqnn_model = seqnn.SeqNN(params_model)
print("built")

seqnn_model.restore(model_file, head_i=head_i)    # model with the mouse head
print("restored")


In [ ]:
hic_diags = params_model["diagonal_offset"]
try:
    target_crop = params_model["trunk"][-2]["cropping"]
except:
    target_crop = params_model["target_crop"]

print("hic_diags: ", hic_diags) 
print("target_crop: ", target_crop)
print("seq_length: ", seq_length)

target_length_cropped = int((seq_length//2048 - target_crop*2 - hic_diags) * ((seq_length//2048 - target_crop*2 - hic_diags) +1)/2) 
target_map_size = seq_length//2048  - target_crop*2 
triu_tup = np.triu_indices(target_map_size, 2)    # Return the indices for the upper-triangle of an (n, m) array, here k=2 (diagonal offset)
# target_map_size, target_length_cropped, triu_tup[0].shape

print("target_length_cropped: ", target_length_cropped)
print("target_map_size: ", target_map_size)
print("shape of triu_tup[0]: ", triu_tup[0].shape)

In [ ]:
background_file = base_dir + '../analysis/background_seqs.fa'
background_seqs = []
with open(background_file,'r') as f:
  for line in f.readlines():
    if '>' in line: continue
    background_seqs.append(dna_io.dna_1hot(line.strip())) 

In [ ]:
spacer_bp = 0
num_inserts = 6
# multi_insert_length = num_inserts * (insert_length+spacer_bp)

# offsets = []
# for i in range(num_inserts):
#     offsets.append( seq_length//2 - multi_insert_length//2 + i * (insert_length+spacer_bp))
# offsets, seq_length//2

In [ ]:
all_inserts = []

for k in range(190,200):
    SINEB2_seq = Seq(genome_open.fetch(new["chrom"][k], new["start"][k], new["end"][k]).upper())
    insert_length = len(SINEB2_seq)
    multi_insert_length = num_inserts * (insert_length+spacer_bp)

    offsets = []
    for i in range(num_inserts):
        offsets.append( seq_length//2 - multi_insert_length//2 + i * (insert_length+spacer_bp))
    
    for background_seq in background_seqs[0:1]:
        seq_1hot = background_seq.copy()
        seq_1hot_motif = dna_io.dna_1hot(genome_open.fetch(new["chrom"][k], new["start"][k], new["end"][k]).upper())
    
    if new["strand"][k] == '-': seq_1hot_motif = dna_io.hot1_rc(seq_1hot_motif)
    for offset in offsets:
        seq_1hot[offset:offset+insert_length] = seq_1hot_motif
    all_inserts.append(seq_1hot)
all_inserts = np.array(all_inserts)

In [ ]:
# all_inserts = []
# i = 3220
# for background_seq in background_seqs[0:1]:
#   seq_1hot = background_seq.copy()
#   seq_1hot_motif = dna_io.dna_1hot(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i] + Position - flank, SINEB2["start"][i] + Position + 19 + flank).upper())
#   # seq_1hot_motif = dna_io.dna_1hot(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i], SINEB2["end"][i]).upper())
#   if SINEB2["strand"][i] == '-': seq_1hot_motif = dna_io.hot1_rc(seq_1hot_motif)
#   for offset in offsets:
#     # print(offset)
#     # print(insert_length)
#     seq_1hot[offset:offset+insert_length] = seq_1hot_motif
#   all_inserts.append(seq_1hot)
# all_inserts = np.array(all_inserts)

In [ ]:
all_inserts.shape

In [ ]:
# for i in range(len(chrom)):
#     if i == 0:
#         mouse_frag = dna_io.dna_1hot(genome_open.fetch(chrom[i], start[i], start[i] + chunk).upper())
#     else:
#         new_frag =  dna_io.dna_1hot(genome_open.fetch(chrom[i], start[i], start[i] + chunk).upper())
#         mouse_frag = np.stack((mouse_frag, new_frag), axis=0)

In [ ]:
# mouse_frag, mouse_frag.shape

In [ ]:
pred = seqnn_model.predict(all_inserts)
                           # , batch_size=10)   # so 20/10 = 2 batches

In [ ]:
# with_retro = dna_io.dna_1hot(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i] - half, SINEB2["end"][i] + half - 212).upper())
# len(with_retro)

In [ ]:
# without_retro = dna_io.dna_1hot(genome_open.fetch(SINEB2["chrom"][i], SINEB2["start"][i] - half, SINEB2["start"][i]).upper()
# + genome_open.fetch(SINEB2["chrom"][i], SINEB2["end"][i], SINEB2["end"][i] + half).upper())
# len(without_retro)

In [ ]:
# mouse_frag = np.stack((with_retro, without_retro), axis=0)
# mouse_frag.shape

In [ ]:
# pred = seqnn_model.predict(mouse_frag)

In [ ]:
pred.shape

In [ ]:
targets = ["mESC", "mESC", "cortical neuron", "neocortex cortical neuron", "neural progenitor cell", "neocortex neural progenitor cell"]
# targets = ["HFF", "H1hESC", "GM12878", "IMR90", "HCT116"]

In [ ]:

# plt.figure(figsize=(8,8))

# for j in range(pred.shape[-1]):
#     # plt.figure(figsize=(pred.shape[0]*5,pred.shape[-1]*5))
#     plt.figure(figsize=(8,8))
#     target_ind = j
#     vlim = .5
#     bin_mid = target_map_size//2
#     window = 50

#     for i in range(len(pred)):
#         insert_pred = pred[:,:,i]
#         # print(i, np.sqrt( (insert_pred**2).sum(axis=0)))

#         plt.subplot(pred.shape[0], 2, i+1)
#         plt.axis("off")
#         im = plt.matshow(
#                 from_upper_triu(  
#                 insert_pred, target_map_size, hic_diags),
#                 vmin=-1*vlim, vmax=vlim, fignum=False, cmap="RdBu_r")
#         plt.colorbar(im, fraction=0.046, pad=0.04)
#         plt.title("scd: " + str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean()) + "\n"
#                  f"target: {targets[j]}")
#             #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
#     plt.tight_layout()
#     plt.show()

In [ ]:
# plt.figure(figsize=(3*5,2*5))

# target_ind = 0
# vlim = .5
# bin_mid = target_map_size//2
# window = 50

# for i in range(pred.shape[-1]):
#     insert_pred = pred[0,:,i]
#     print(i, np.sqrt( (insert_pred**2).sum(axis=0)))

#     plt.subplot(2,3, i+1)
#     im = plt.matshow(
#             from_upper_triu(  
#             insert_pred, target_map_size,hic_diags),
#             vmin=-1*vlim, vmax=vlim, fignum=False,cmap='RdBu_r')
#     plt.colorbar(im, fraction=0.046, pad=0.04)
#     # plt.title('genomic-scd: '+str(seq_coords_df['genomic_SCD'].values[i]) +'\n'+
#     #           'insert-scd: '+str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean() ) 
#     #           ) 
#     plt.title('insert-scd: '+str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean() ) + "\n" + f"target: {targets[i]}"
#     )
#     #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
# plt.tight_layout()
# plt.show()

In [ ]:
# scd_score = []

In [ ]:
plt.figure(figsize=(5*3,2*3))

target_ind = 0
vlim = .5
bin_mid = target_map_size//2
window = 50
for i in range(10):
    insert_pred = pred[i]
    print(i, np.sqrt( (insert_pred**2).sum(axis=0)))
    scd_score.append(np.sqrt( (insert_pred**2).sum(axis=0)  ).mean())
    
    plt.subplot(2,5, i+1)
    im = plt.matshow(
            from_upper_triu(  
            insert_pred[:,target_ind], target_map_size,hic_diags),
            vmin=-1*vlim, vmax=vlim, fignum=False,cmap='RdBu_r')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.title('insert-scd: '+str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean() ) 
              ) 
    #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
plt.tight_layout()
plt.show()

In [ ]:
len(scd_score)

In [ ]:
import copy
scd_copy = copy.copy(scd_score)

In [ ]:
dfm = pd.DataFrame(scd_copy, columns =['scd'], index=[i for i in range(len(scd_copy))])
with_scd = new.join(dfm, how="left")

In [ ]:
with_scd

In [ ]:
with_scd.to_csv("./200_SINEB2s.csv")

In [ ]:
plt.figure(figsize=(10,8))
# plt.scatter(x=with_scd.scd, y=with_scd.score, s=with_scd.score*5, alpha=0.5, c=(abs(with_scd.end - with_scd.start)))
plt.scatter(x=with_scd.scd, y=with_scd.aln_score, s=with_scd.score*5, alpha=0.5, c=with_scd.score)
plt.colorbar()
plt.xlabel("SCD")
plt.ylabel("Alignment score")
plt.show()

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
 
# Creating figure
fig = plt.figure(figsize = (15, 12))
ax = plt.axes(projection ="3d")

sctt = ax.scatter3D(with_scd.scd, with_scd.aln_score, with_scd.score, s=70, c= with_scd.length)
 
ax.set_xlabel('SCD')
ax.set_ylabel('Alignment Score')
ax.set_zlabel('CTCF Score')

fig.colorbar(sctt, shrink = 0.5, aspect = 12)

ax.view_init(30, 250)

plt.show()

In [ ]:

# for j in range(pred.shape[-1]):
#     plt.figure(figsize=(pred.shape[0]*6,pred.shape[-1]*5))
#     target_ind = j
#     vlim = .5
#     bin_mid = target_map_size//2
#     window = 50

#     for i in range(len(pred)):
#         insert_pred = pred[i]
#         # print(i, np.sqrt( (insert_pred**2).sum(axis=0)))

#         plt.subplot(6, 2, i+1)
#         plt.axis("off")
#         im = plt.matshow(
#                 from_upper_triu(  
#                 insert_pred[:,target_ind], target_map_size, hic_diags),
#                 vmin=-1*vlim, vmax=vlim, fignum=False, cmap="RdBu_r")
#         plt.colorbar(im, fraction=0.046, pad=0.04)
#         plt.title("scd: " + str(  np.sqrt( (insert_pred[:,:,j]**2).sum(axis=0)  ).mean()) + "\n"
#                  f"target: {targets[j]}")
#             #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
#     plt.tight_layout()
#     plt.show()

In [ ]:
# flanks = ["1x", "4x", "6x", "0-CTCF", "10", "20", "30", "40", "50"]
# scores = [10.76, 15.23, 19.23, 15.22, 20.61, 21.97, 19.39, 20.42, 19.]

# df = pd.DataFrame(list(zip(flanks, scores)), columns =['flanks', 'scores'])

In [ ]:
# plt.figure(figsize=(12,8))
# # sns.histplot(data=score_list)

# ax = sns.barplot(data=df, x="flanks", y="scores")
# ax.set(xlabel='variant', ylabel='score')
# # plt.savefig("SINEB2_scores_hist.png")
# plt.show()